<a href="https://colab.research.google.com/github/chaitanyap1/Datascience-projects/blob/master/Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
copied_path = '/content/drive/My Drive/P3 Quora Text Classification with embeddings/train.csv'
question = pd.read_csv(copied_path)

In [5]:
question.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [6]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

Using TensorFlow backend.


In [0]:
embedding_index ={}

f = open('/content/drive/My Drive/glove.42B.300d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype ='float32')
  embedding_index[word] = coefs
f.close()

In [9]:
print(question.shape)
question.head()

(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [10]:
X = question['question_text']
Y = question['target']
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1044897,), (1044897,), (261225,), (261225,))

In [11]:
y_test.value_counts()

0    245102
1     16123
Name: target, dtype: int64

In [12]:
class_weight = class_weight.compute_class_weight('balanced', 
                                                 np.unique(y_train), 
                                                    y_train)
print(class_weight)
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

[0.5329965  8.07656098]


{0: 0.5329965007498393, 1: 8.076560978249107}

In [13]:
x_train

771247     Why do Pakistani commuter bikes have old desig...
623410     Do non visa, green card or other illegal worke...
139604     Do you think someone does themselves a service...
1293300    Where can I read about Democrat plans to manag...
1131214                   How do I meet gay men from bilbao?
                                 ...                        
736895     What are the best ways to help a confabulator ...
33590      What do you do when someone plays a practical ...
510765     Is pulling a trigger on yourself in the head p...
251037     Are varginas supposed to be pink or red inside...
488022     How do you control your mind and your actions ...
Name: question_text, Length: 1044897, dtype: object

In [14]:
import nltk
nltk.download('punkt')  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [16]:
print(np.percentile(sent_lens,99))
np.percentile(sent_lens,95)

45.0


31.0

In [17]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [0]:
seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

In [19]:
vocab_size=len(tk.word_index)
vocab_size

196288

In [20]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 45), (261225, 45))

In [0]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [0]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [25]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 45, 300)           58886700  
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               3289

In [0]:
filepath='/content/drive/My Drive/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [0]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5328573351855138, 1: 8.108651115146436},
              batch_size=10000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])



Train on 1044897 samples, validate on 261225 samples
Epoch 1/10
1044897/1044897 [==============================] - 870s 833us/step - loss: 0.3474 - acc: 0.8387 - val_loss: 0.3147 - val_acc: 0.8749

Epoch 00001: val_loss improved from inf to 0.31472, saving model to /content/drive/My Drive/weights-01-0.3147.h5
Epoch 2/10
1044897/1044897 [==============================] - 848s 811us/step - loss: 0.2856 - acc: 0.8807 - val_loss: 0.2698 - val_acc: 0.8832

Epoch 00002: val_loss improved from 0.31472 to 0.26981, saving model to /content/drive/My Drive/weights-02-0.2698.h5
Epoch 3/10
1044897/1044897 [==============================] - 823s 787us/step - loss: 0.2647 - acc: 0.8893 - val_loss: 0.2380 - val_acc: 0.8980

Epoch 00003: val_loss improved from 0.26981 to 0.23804, saving model to /content/drive/My Drive/weights-03-0.2380.h5
Epoch 4/10
1044897/1044897 [==============================] - 821s 786us/step - loss: 0.2480 - acc: 0.8947 - val_loss: 0.2195 - val_acc: 0.9087

Epoch 00004: val_l

In [0]:
best_model = load_model('/content/drive/My Drive/weights-09-0.1849.h5')

In [38]:
best_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 45, 300)           58886700  
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               3289

In [40]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9637618014028762